In [1]:
import collections
import math
import os
import shutil
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
import torchvision
from d2l import torch as d2l
from PIL import Image
from torch import nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision.models as models
from sklearn.model_selection import train_test_split
import time

In [2]:
labels_dataframe = pd.read_csv("./dataset/cifar-10/trainLabels.csv")
# labels_dataframe.head(5)

In [3]:
leaves_labels = sorted(list(set(labels_dataframe["label"])))
n_classes = len(leaves_labels)
n_classes

10

In [4]:
cls_to_num = dict(zip(leaves_labels, range(len(leaves_labels))))
num_to_cls = dict(zip(range(len(leaves_labels)), leaves_labels))

In [5]:
class TrainData(Dataset):
    def __init__(self, csv_path, img_path, transform):
        """
        Args:
            csv_path (string): csv 文件路径
            img_path (string): 图像文件所在路径
            mode (string): 训练模式还是测试模式
            valid_ratio (float): 验证集比例
        """
        self.img_path = img_path
        self.data_info = pd.read_csv(csv_path)
        self.data_len = len(self.data_info)
        self.train_img = np.asarray(self.data_info.iloc[: self.data_len, 0])
        self.train_label = np.asarray(self.data_info.iloc[: self.data_len, 1])
        self.transform = transform
        self.img_arr = self.train_img
        self.label_arr = self.train_label

    def __getitem__(self, index):
        img_name = self.img_arr[index]
        img_as_img = Image.open(os.path.join(self.img_path, f"{img_name}.png"))
        img_as_tensor = self.transform(img_as_img)
        label = self.label_arr[index]
        num_label = cls_to_num[label]
        return img_as_tensor, num_label

    def __len__(self):
        return self.data_len

In [7]:
transform_train = torchvision.transforms.Compose(
    [
        # 在高度和宽度上将图像放大到40像素的正方形
        torchvision.transforms.Resize(40),
        # 随机裁剪出一个高度和宽度均为40像素的正方形图像，
        # 生成一个面积为原始图像面积0.64到1倍的小正方形，
        # 然后将其缩放为高度和宽度均为32像素的正方形
        torchvision.transforms.RandomResizedCrop(
            32, scale=(0.64, 1.0), ratio=(1.0, 1.0)
        ),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        # 标准化图像的每个通道
        torchvision.transforms.Normalize(
            [0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010]
        ),
    ]
)
transform_test = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            [0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010]
        ),
    ]
)

In [8]:
train_path = "dataset/cifar-10/trainLabels.csv"
train_img_path = "dataset/cifar-10/train/"

In [9]:
train_ids, valid_ids, train_labels, valid_labels = train_test_split(
    labels_dataframe["id"],
    labels_dataframe["label"],
    test_size=0.1,
    shuffle=True,
    random_state=42,
)

In [10]:
len(train_ids), len(valid_ids)

(45000, 5000)

In [11]:
train_set = TrainData(
    csv_path=train_path, img_path=train_img_path, transform=transform_train
)

In [12]:
train_subsampler = torch.utils.data.SubsetRandomSampler(np.array(train_ids))
valid_subsampler = torch.utils.data.SubsetRandomSampler(np.array(valid_ids))
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=1,
    sampler=train_subsampler,
    num_workers=4,
    pin_memory=False,
    drop_last=True
)
valid_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=128,
    sampler=valid_subsampler,
    num_workers=4,
    pin_memory=False,
    drop_last=True
)

In [13]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        model = model
        for param in model.parameters():
            param.requires_grad = False
# Resnet模型
def resnet_model(num_classes,feature_extracting=False):
    model=models.resnet18(pretrained=False)
    set_parameter_requires_grad(model,feature_extracting)
    num_ftrs=model.fc.in_features
    model.fc=nn.Sequential(nn.Linear(num_ftrs,num_classes))
    return model

In [14]:
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        nn.init.xavier_uniform_(m.weight)

In [15]:
batch_size=128
device, num_epochs, lr, wd = 'cuda', 2, 2e-4, 5e-4
lr_period, lr_decay = 4, 0.9

In [16]:
loss_function = nn.CrossEntropyLoss()
model = resnet_model(10)
model.apply(init_weights)
model = model.to(device)
model.device = device
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
# 线性衰减
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, lr_period, lr_decay)

In [ ]:
best_acc=0.
model_path='./models/tiny_cifar.pth'
for epoch in range(num_epochs):
    # time.sleep(0.5)
    model.train()
    print(f'Starting epoch {epoch+1}')
    print('*'*30)
    train_loss = []
    train_acc = []
    for batch in tqdm(train_loader):
        time.sleep(0.005)
        imgs,labels=batch
        imgs,labels=imgs.to(device),labels.to(device)
        logits=model(imgs)
        loss = loss_function(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # scheduler.step()
        train_loss.append(loss.item())         
    print("第%d个epoch的学习率：%f" % (epoch+1,optimizer.param_groups[0]['lr']))
    scheduler.step()
    train_avg_loss = np.sum(train_loss) / len(train_loss)
    print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_avg_loss:.5f}")
    print('Training process has finished. Saving trained model.')
    print('Starting validation')
    model.eval()
    valid_loss = []
    valid_acc = []
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            time.sleep(.005)
            imgs,labels=batch
            imgs,labels=imgs.to(device),labels.to(device)
            logits=model(imgs)
            loss = loss_function(logits, labels)
            valid_loss.append(loss.cpu().item())
            valid_acc.append((logits.argmax(dim=-1)==labels).float().mean().cpu())
        valid_avg_loss=np.sum(valid_loss)/len(valid_loss)
        valid_avg_acc=np.sum(valid_acc)/len(valid_acc)
        print(f"[ Valid | {epoch + 1:03d}/{num_epochs:03d} ] loss = {valid_avg_loss:.5f}, acc = {valid_avg_acc:.5f}")
        if valid_acc > best_acc:
            best_acc = valid_acc
            torch.save(model.state_dict(), model_path)
            print('saving model with acc {:.3f}'.format(best_acc))

Starting epoch 1
******************************


  0%|                                                                                        | 0/50000 [00:00<?, ?it/s]

In [17]:
li = []
for name in os.listdir('dataset/cifar-10/test'):
    li.append(name)
img_arr=np.array(li)
img_arr[:10]

array(['1.png', '10.png', '100.png', '1000.png', '10000.png',
       '100000.png', '100001.png', '100002.png', '100003.png',
       '100004.png'], dtype='<U10')

In [18]:
class TestData(Dataset):
    def __init__(self, transform):
        """
        Args:
            csv_path (string): csv 文件路径
            img_path (string): 图像文件所在路径
            mode (string): 训练模式还是测试模式
            valid_ratio (float): 验证集比例
        """
        # self.img_path = img_path
        self.transform = transform
        # li = []
        # for name in os.listdir(img_path):
        #     li.append(name)
        self.img_arr = img_arr
        self.data_len = len(self.img_arr)

    def __getitem__(self, index):
        img_name = self.img_arr[index]
        img_as_img = Image.open(os.path.join('dataset/cifar-10/test', img_name))
        img_as_tensor = self.transform(img_as_img)
        return img_as_tensor

    def __len__(self):
        return self.data_len

In [20]:
test_set=TestData(transform=transform_test)

In [21]:
test_loader= torch.utils.data.DataLoader(test_set, batch_size=128, 
                                         drop_last=False,num_workers=0)

In [27]:
device='cuda'
model = resnet_model(10)
model = model.to(device)
model.device=device
model_path='models/tiny_cifar.pth'
model.load_state_dict(torch.load(model_path))
model.eval()  #batchnormize用全局的batch算
preds = []
# img_names=[]
for batch in tqdm(test_loader):
    imgs = batch
    imgs = imgs.to(device)
    # img_names.append(name)
    with torch.no_grad():
        logits = model(imgs)
    preds.extend(logits.argmax(dim=-1).cpu().numpy().tolist())
labels=[num_to_cls[pred] for pred in preds]
num_arr=[img_name.split('.')[0] for img_name in img_arr]
df = pd.DataFrame({'id': num_arr, 'label': labels})
df.to_csv('dataset/cifar-10/submission.csv',index=False)

100%|██████████████████████████████████████████████████████████████████████████████| 2344/2344 [20:09<00:00,  1.94it/s]


['1',
 '10',
 '100',
 '1000',
 '10000',
 '100000',
 '100001',
 '100002',
 '100003',
 '100004',
 '100005',
 '100006',
 '100007',
 '100008',
 '100009',
 '10001',
 '100010',
 '100011',
 '100012',
 '100013',
 '100014',
 '100015',
 '100016',
 '100017',
 '100018',
 '100019',
 '10002',
 '100020',
 '100021',
 '100022',
 '100023',
 '100024',
 '100025',
 '100026',
 '100027',
 '100028',
 '100029',
 '10003',
 '100030',
 '100031',
 '100032',
 '100033',
 '100034',
 '100035',
 '100036',
 '100037',
 '100038',
 '100039',
 '10004',
 '100040',
 '100041',
 '100042',
 '100043',
 '100044',
 '100045',
 '100046',
 '100047',
 '100048',
 '100049',
 '10005',
 '100050',
 '100051',
 '100052',
 '100053',
 '100054',
 '100055',
 '100056',
 '100057',
 '100058',
 '100059',
 '10006',
 '100060',
 '100061',
 '100062',
 '100063',
 '100064',
 '100065',
 '100066',
 '100067',
 '100068',
 '100069',
 '10007',
 '100070',
 '100071',
 '100072',
 '100073',
 '100074',
 '100075',
 '100076',
 '100077',
 '100078',
 '100079',
 '10008',
